In [1]:
import mysql.connector
import pandas as pd
import unicodedata
import numpy as np

# <center>Modelo ER</center>

CREATE DATABASE QUANTYLE_ANALITICS;


use QUANTYLE_ANALITICS;


# --------------------------------------------------------------------


CREATE TABLE state (
    state_id INT PRIMARY KEY AUTO_INCREMENT NOT NULL,
    state VARCHAR(20)
);


CREATE TABLE user_yelp (
    user_id VARCHAR(100) PRIMARY KEY NOT NULL,
    name VARCHAR(100),
    creation TIMESTAMP,
    review_count INT,
    influence INT,
    stars DECIMAL(4, 2)
);


CREATE TABLE business_yelp (
    business_id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(300),
    latitude DECIMAL(10, 7),
    longitude DECIMAL(10, 7),
    avg_stars DECIMAL(4, 2),
    state_id INT,
    FOREIGN KEY (state_id) REFERENCES state(state_id)
);


CREATE TABLE reviews_yelp (
    review_id VARCHAR(100) PRIMARY KEY,
    user_id VARCHAR(100),
    business_id VARCHAR(100),
    stars DECIMAL(4, 2),
    sentiment INT,
    date VARCHAR(200)#,
    #FOREIGN KEY (user_id) REFERENCES user_yelp(user_id),
    #FOREIGN KEY (business_id) REFERENCES business_yelp(business_id)
);


# ----------------------------------------------------------------


CREATE TABLE business_google (
    gmap_id VARCHAR(100) PRIMARY KEY NOT NULL,
    name VARCHAR(300),
    latitude DECIMAL(10, 7), 
    longitude DECIMAL(10, 7), 
    avg_stars DECIMAL(4, 2), 
    state_id INT,
    FOREIGN KEY (state_id) REFERENCES state(state_id)
);


CREATE TABLE user_google (
	user_id VARCHAR(300) NOT NULL,
    name VARCHAR(300),
    creation DATETIME,
    review_count INT,
    stars DECIMAL(4, 2),
    PRIMARY KEY (user_id)
);


CREATE TABLE reviews_google (
    review_id VARCHAR(200),
    user_id VARCHAR(300) NOT NULL,
    gmap_id VARCHAR(100),
    stars DECIMAL(4, 2),
    sentiment INT,
    date TIMESTAMP,
    resp_sentiment INT,
    resp_date TIMESTAMP
);

# --------------------------------------------------------------------


CREATE TABLE categories (
    categories_id INT PRIMARY KEY,
    name VARCHAR(100)
);


CREATE TABLE categories_google (
    gmap_id VARCHAR(100),
    categories_id INT,
    FOREIGN KEY (categories_id) REFERENCES categories(categories_id),
    FOREIGN KEY (gmap_id) REFERENCES business_google(gmap_id)
);


CREATE TABLE categories_yelp (
    business_id VARCHAR(100),
    categories_id INT,
    FOREIGN KEY (categories_id) REFERENCES categories(categories_id),
    FOREIGN KEY (business_id) REFERENCES business_yelp(business_id)
);




# SHOW VARIABLES LIKE 'secure_file_priv';


---

<h3><center>Funciones</center></h3>

In [11]:
'''def conectar_base_oficial():
    try:
        conexion = mysql.connector.connect(
            host='quantyle.cxqm3ttktdpq.us-east-2.rds.amazonaws.com',
            user='admin',
            password='1533542415',
            database='quantyle'
        )
        print("¡Conexión establecida correctamente!")
        return conexion
    except mysql.connector.Error as error:
        print("Error al conectarse a MySQL:", error)
        return None'''

In [2]:
def conectar_mysql():
    try:
        conexion = mysql.connector.connect(
            host='localhost',
            user='root',
            password='1533542415',
            database='quantyle_analitics'
        )
        print("¡Conexión establecida correctamente!")
        return conexion
    except mysql.connector.Error as error:
        print("Error al conectarse a MySQL:", error)
        return None
    

---

# <center>Comienzo de la carga de datos</center>

In [3]:
# Establecer la conexión con la base de datos
conexion = conectar_mysql()

# Verificar si la conexión se realizó con éxito
if conexion.is_connected():
    print("Conexión exitosa")

# Realizar operaciones en la base de datos, por ejemplo, ejecutar una consulta SQL
cursor = conexion.cursor()

# Ejemplo de consulta SQL
consulta = "SELECT * FROM categories_yelp"

# Ejecutar la consulta
cursor.execute(consulta)

# Obtener los resultados
resultados = cursor.fetchall()

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()

¡Conexión establecida correctamente!
Conexión exitosa


In [4]:
# Establecer la conexión con la base de datos
conexion = conectar_mysql()

# Verificar si la conexión se realizó con éxito
if conexion.is_connected():
    print("Conexión exitosa")

# Realizar operaciones en la base de datos, por ejemplo, ejecutar una consulta SQL
cursor = conexion.cursor()

# Ejemplo de consulta SQL
consulta = "SELECT * FROM categories_yelp"

# Ejecutar la consulta
cursor.execute(consulta)

# Obtener los resultados
resultados = cursor.fetchall()

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()


¡Conexión establecida correctamente!
Conexión exitosa


# 1) State

In [20]:
state = pd.read_parquet(r'..\datasets\processed\bd\1_states.parquet.gz')

conexion = conectar_mysql()

# Insertar filas en la tabla state
cursor = conexion.cursor()
for index, row in state.iterrows():
    cursor.execute("INSERT INTO state (state_id, state) VALUES (%s, %s)", (row['state_id'],row['state']))  # Asegúrate de tener la coma al final

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 2) Categories

In [22]:
categories = pd.read_parquet(r'..\datasets\processed\bd\2_categories.parquet.gz')

# Aplicar strip() a la columna 'name' para eliminar espacios en blanco adicionales
categories['name'] = categories['name'].str.strip()

# Convertir la columna 'name' a minúsculas para uniformizar los datos
categories['name'] = categories['name'].str.lower()

# Eliminar las filas duplicadas basadas en la columna 'name'
categories.drop_duplicates(subset=['name'], keep='first', inplace=True)

conexion = conectar_mysql()

# Insertar filas en la tabla categories
cursor = conexion.cursor()
for index, row in categories.iterrows():
    cursor.execute("INSERT INTO categories (categories_id , name) VALUES (%s, %s)", (row['categories_id'], row['name'],))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 3) User yelp

In [24]:
user_yelp = pd.read_parquet(r'..\datasets\processed\bd\3_user_yelp.parquet.gz')

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_yelp.iterrows():
    cursor.execute("INSERT INTO user_yelp (user_id, name, creation, review_count, influence, stars) VALUES (%s, %s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['influence'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 4) User google

In [49]:
user_google = pd.read_parquet(r'..\datasets\processed\bd\4_user_google.parquet.gz')

user_google.replace(' ', pd.NA, inplace=True)

user_google['name'] = user_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)

user_google.drop(user_google[user_google['name'] == ''].index, inplace=True)

user_google.dropna(inplace=True)

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_google.iterrows():
    cursor.execute("INSERT INTO user_google (user_id, name, creation, review_count, stars) VALUES (%s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 5) Bussiness google

In [51]:
business_google = pd.read_parquet(r'..\datasets\processed\bd\5_business_google.parquet.gz')


business_google['name'] = business_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)


conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_google[['gmap_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_google (gmap_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 6) Business yelp

In [54]:
business_yelp = pd.read_parquet(r'..\datasets\processed\bd\6_business_yelp.parquet.gz')
business_yelp['name'] = business_yelp['name'].replace(r'[^a-zA-Z\s]', '', regex=True)

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_yelp[column] = business_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_yelp[['business_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_yelp (business_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 7) Categories google

In [55]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_google[column] = categories_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)
    
conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_google[['gmap_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_google (gmap_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 8) Categories yelp

In [56]:
categories_yelp = pd.read_parquet(r'..\datasets\processed\bd\8_categories_yelp.parquet.gz')

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_yelp[column] = categories_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_yelp[['business_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_yelp (business_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 9) Reviews google

In [12]:
reviews_google = pd.read_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz')

# export
#reviews_google.to_csv(r'reviews_google.csv',encoding='utf-8',index=False)


LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_google.csv'
INTO TABLE reviews_google
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, gmap_id, stars, sentiment, date, resp_sentiment, resp_date);


# 10) Reviews yelp

In [ ]:
reviews_yelp = pd.read_parquet(r'..\datasets\processed\bd\10_reviews_yelp.parquet.gz')

reviews_yelp['date'] = reviews_yelp['date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')

#reviews_yelp.to_csv(r'reviews_yelp.csv',encoding='utf-8', index=False)



### Query de carga para el paso 9 y 10

In [ ]:
'''LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_google.csv'
INTO TABLE reviews_google
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, gmap_id, stars, sentiment, date, resp_sentiment, resp_date);





LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_yelp.csv'
INTO TABLE reviews_yelp
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, business_id, stars, sentiment, date);'''




# Population_data

In [59]:

# Crear el DataFrame original con los datos proporcionados
data = {
    '2': [39144818, 39250017, 39399349, 39538223, 39613506, 39538223, 39296476, 39056079, 38982847],
    '1': [20271272, 20656589, 20984400, 21299325, 21477737, 21538187, 21733312, 22244823, 22733312],
    '3': [12859995, 12802503, 12778828, 12741080, 12671821, 12812508, 12671869, 12518144, 12470000],
    '4': [8958013, 9005644, 9032872, 8908520, 8882190, 9288994, 9261692, 9288994, 9290000]
}

index = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

df = pd.DataFrame(data, index=index)

# Resetear el índice para convertir los años en una columna 'Date'
df.reset_index(inplace=True)
df.rename(columns={'index': 'Date'}, inplace=True)

# Convertir el DataFrame a un formato largo (usando melt) para combinar las columnas de población en una sola columna
df = df.melt(id_vars='Date', var_name='state_id', value_name='Population')

df.head()

,Date,state_id,Population
0,2015,2,39144818
1,2016,2,39250017
2,2017,2,39399349
3,2018,2,39538223
4,2019,2,39613506


In [60]:
conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in df[['state_id','Date','Population']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO Population_data (state_id, date, population) VALUES (%s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()

¡Conexión establecida correctamente!


# Trigger

use quantyle;

DROP TABLE IF EXISTS new_uploads;
CREATE TABLE new_uploads (
    id INT AUTO_INCREMENT PRIMARY KEY,
    tipo_tabla VARCHAR(100),
    registro_id VARCHAR(100),
    fecha_creacion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


DROP TRIGGER IF EXISTS after_insert_business_trigger;
DROP TRIGGER IF EXISTS after_insert_reviews_trigger;
DROP TRIGGER IF EXISTS after_insert_users_trigger;


DELIMITER //

CREATE TRIGGER after_insert_business_trigger
AFTER INSERT ON business_yelp
FOR EACH ROW
BEGIN
    INSERT INTO new_uploads (tipo_tabla, registro_id)
    VALUES ('business_yelp', NEW.business_id);
END;
//

CREATE TRIGGER after_insert_reviews_trigger
AFTER INSERT ON reviews_yelp
FOR EACH ROW
BEGIN
    INSERT INTO TablaAuxiliar (tipo_tabla, registro_id)
    VALUES ('reviews_yelp', NEW.review_id);
END;
//

CREATE TRIGGER after_insert_users_trigger
AFTER INSERT ON user_yelp
FOR EACH ROW
BEGIN
    INSERT INTO TablaAuxiliar (tipo_tabla, registro_id)
    VALUES ('user_yelp', NEW.user_id);
END;
//

DELIMITER ;

